In [1]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import keras 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
batch_size = 32
image_size = (150, 150)  # Adjust the size as needed

data_dir='C:/National college/Foundation of artificial intelligent/project/TB_Chest_Radiography_Database'
train=keras.utils.image_dataset_from_directory(data_dir,image_size=image_size,
                                                validation_split=0.1,
                                                label_mode='categorical',
                                                subset='training',seed=123)
val=keras.utils.image_dataset_from_directory(data_dir,image_size=image_size,
                                             label_mode = 'categorical',
                                                validation_split=0.2,
                                                subset='validation',seed=123)

Found 4200 files belonging to 2 classes.
Using 3780 files for training.
Found 4200 files belonging to 2 classes.
Using 840 files for validation.


In [3]:
train=train.map(lambda x,y:(x/255,y))
val=val.map(lambda x,y:(x/255,y))

In [4]:
VGG_model = Sequential()

pretrained_model= keras.applications.VGG16(include_top=False,
                   input_shape=(150,150,3),
                   pooling='max',classes=4,
                   weights='imagenet')


VGG_model.add(pretrained_model)
VGG_model.add(Flatten())
VGG_model.add(Dense(512, activation='relu'))
VGG_model.add(BatchNormalization())  # Batch Normalization layer
VGG_model.add(Dropout(0.5))

VGG_model.add(Dense(2, activation='softmax'))
pretrained_model.trainable=False


In [5]:
VGG_model.compile(optimizer=keras.optimizers.Adam(0.0001),
              loss=keras.losses.CategoricalCrossentropy(),metrics=["accuracy"])


In [6]:
VGG_model.fit(train,epochs=1,
                 validation_data=val)




119/119 [==============================] - 321s 3s/step - loss: 0.5835 - accuracy: 0.7479 - val_loss: 0.2226 - val_accuracy: 0.9429


In [7]:
VGG_model.save("C:/National college/Artificial Intelligence Driven Decision Making (MSCAI1)/tuberculosis.h5")


C:\Users\Waqas\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# Evaluate the model on the validation set
loss, accuracy = VGG_model.evaluate(val)

# Print the evaluation metrics
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

# Calculate precision, recall, and F1-score
y_pred = VGG_model.predict(val)
y_pred_class = np.argmax(y_pred, axis=1)
y_true = np.concatenate([y for x, y in val], axis=0)
y_true_class = np.argmax(y_true, axis=1)

precision = precision_score(y_true_class, y_pred_class)
recall = recall_score(y_true_class, y_pred_class)
f1 = f1_score(y_true_class, y_pred_class)

print("Validation Precision:", precision)
print("Validation Recall:", recall)
print("Validation F1-score:", f1)

# Confusion matrix
conf_mat = confusion_matrix(y_true_class, y_pred_class)
print("Confusion Matrix:")
print(conf_mat)

# Classification report
print("Classification Report:")
print(classification_report(y_true_class, y_pred_class))

27/27 [==============================] - 50s 2s/step - loss: 0.2226 - accuracy: 0.9429
Validation Loss: 0.2225508689880371
Validation Accuracy: 0.9428571462631226
27/27 [==============================] - 53s 2s/step
Validation Precision: 0.17142857142857143
Validation Recall: 0.12244897959183673
Validation F1-score: 0.14285714285714288
Confusion Matrix:
[[606  87]
 [129  18]]
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.87      0.85       693
           1       0.17      0.12      0.14       147

    accuracy                           0.74       840
   macro avg       0.50      0.50      0.50       840
weighted avg       0.71      0.74      0.73       840

